## Data for stadium building problem

In [11]:
using JuMP

tasks = 1:18
durations = [2 16 9 8 10 6 2 2 9 5 3 2 1 7 4 3 9 1]
predecessors = ( [], [1], [2], [2], [3], [4,5], [4], [6], [4,6], [4], [6], [9], [7], [2], [4,14], [8,11,14], [12], [17] )
pred_dict = Dict(zip(tasks,predecessors));   # dictionary mapping tasks --> predecessors.

# additional columns of data (maximum reduction possible )
max_reduction =  [0,  3,  1,  2,  2,  1, 1, 0,  2,  1,  1, 0, 0,  2,  2, 1,  3, 0]  # max reduction (in weeks)
cost_reduction = [0, 30, 26, 12, 17, 15, 8, 0, 42, 21, 18, 0, 0, 22, 12, 6, 16, 0]  # cost of reduction ($1,000/week)
bonus_amount = 30     # bonus for expediting the project ($1,000/week )
;

# a)

# i)
### Mathematical Model

Decision variables:  
Start time of the task $t_i$ for all tasks $i \in \mathcal{T}$, where $\mathcal{T}$ is the set of all tasks.
Note that we define a finish task, which defines the last "task" and has duration 0.

Constraints:  
Each task can only start when the previous task is done,  
$$t_i \geq t_j + d_j\quad \forall j \in  \mathcal{P}_i \quad \forall i \in\mathcal{T}$$
where $\mathcal{P}_i$ is the set of all tasks that must be finished before task $i$ can start (i.e., predecessor tasks) and $d_j$ is the duration of task $j$.

The first task will start at time 0, i.e.,
$$t_1 == 0$$

Objective:  
$$\min_{t} t_x + d_x$$
where $t_x$ and $d_x$ are the start time and duration of the finish task, respectively. 


# ii)

In [35]:
using JuMP,Clp
m = Model(with_optimizer(Clp.Optimizer))

@variable(m, tstart[tasks])

@constraint(m, tstart[:1] == 0) #The firsst task starts at time 0

for i in tasks
    for j in pred_dict[i]
        @constraint(m, tstart[i] >= tstart[j] + durations[j])
    end
end

@objective(m, Min, tstart[:18] + durations[:18])

optimize!(m)
println(value.(tstart))
println("The stadium will take ", objective_value(m), " days to build.")
println()

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:18
And data, a 18-element Vector{Float64}:
  0.0
  2.0
 18.0
 18.0
 27.0
 37.0
 26.0
 43.0
 43.0
 26.0
 43.0
 52.0
 28.0
 18.0
 26.0
 46.0
 54.0
 63.0
The stadium will take 64.0 days to build.

Coin0506I Presolve 0 (-23) rows, 0 (-18) columns and 0 (-45) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 64
Coin0511I After Postsolve, objective 64, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 64 - 0 iterations time 0.002, Presolve 0.00


# b)

## i)

## Data for car rental problem

In [ ]:
position = [0 0; 20 20; 18 10; 30 12; 35 0; 33 25; 5 27; 5 10; 11 0; 2 15]
present = [8; 13; 4; 8; 12; 2; 14; 11; 15; 7]
required = [10; 6; 8; 11; 9; 7; 15; 7; 9; 12]
;

In [10]:
demand = [10   6   8  11   9   7  15   7   9  12]';
stock  = [ 8  13   4   8  12   2  14  11  15   7]';
cost   = 0.50;

xcord  = [ 0  20  18  30  35  33   5   5  11   2]';
ycord  = [ 0  20  10  12   0  25  27  10   0  15]';
n      = length(xcord);

distance = zeros(n,n);
for i=1:n
    for j=1:n
        distance(i,j) = 1.3*sqrt( (xcord(i) - xcord(j))^2 + (ycord(i) - ycord(j))^2);
    end
end

idx_excess = find(stock-demand > 0);
n_excess   = length(idx_excess);

idx_need   = find(stock-demand < 0);
n_need     = length(idx_need);

move = tom('move',n_excess,n_need,'int');

% Bounds
bnds = {0 <= move};

% Excess constraint
con1 = {sum(move,2) == stock(idx_excess) - demand(idx_excess)};

% Need constraint
con2 = {sum(move,1)' == demand(idx_need) - stock(idx_need)};

% Objective
objective = sum(sum(move.*cost.*distance(idx_excess,idx_need)));

constraints = {bnds, con1, con2};
options = struct;
options.solver = 'cplex';
options.name   = 'Car Rental';
sol = ezsolve(objective,constraints,[],options);

PriLev = 1;
if PriLev > 0
    temp = sol.move;
    disp('THE SENDING OF CARS')
    for i = 1:n_excess,       % scan all positions, disp interpretation
        disp(['agency ' num2str(idx_excess(i)) ' sends: ' ])
        for j = 1:n_need,
            if temp(i,j) ~= 0
                disp(['   ' num2str(temp(i,j)) ' car(s) to agency ' ...
                    num2str(idx_need(j))])
            end
        end
        disp(' ')
    end

    disp('THE GETTING OF CARS')
    for j = 1:n_need,
        disp(['agency ' num2str(idx_need(j)) ' gets: ' ])
        for i = 1:n_excess,       % scan all positions, disp interpretation
            if temp(i,j) ~= 0
                disp(['   ' num2str(temp(i,j)) ' car(s) from agency ' ...
                    num2str(idx_excess(i))])
            end
        end
        disp(' ')
    end
end


LoadError: cannot define function distance; it already has a value

## The chess problem

### i)

In [2]:
using Pkg
using JuMP, Clp

m3 = Model()
set_optimizer(m3, with_optimizer(Clp.Optimizer))

#decision variables
@variable(m3, l >= 0)
@variable(m3, s >= 0)

# constraints
@constraint(m3, 2*l+3*s <= 160)
@constraint(m3, 4*l+s <= 200)

#objective
@objective(m3, Max, 8*l + 5*s)


print(m3)
optimize!(m3)
println("Termination status: ", termination_status(m3))


l_value = value(l)
s_value = value(s)

println([l_value, s_value])


Termination status: OPTIMAL
[44.00000000000001, 23.999999999999993]
Coin0506I Presolve 2 (0) rows, 2 (0) columns and 4 (0) elements
Clp0006I 0  Obj 0 Dual inf 13 (2)
Clp0006I 2  Obj 472
Clp0000I Optimal - objective value 472
Clp0032I Optimal objective 472 - 2 iterations time 0.002


In [5]:
# using PyPlot

# figure(figsize=(6, 4))
# l = range(0, stop=6, length=8) |> collect


# plot(l, (160 .- 2*l) / 3)   # first constraint
# plot(l, (200 .- 4*l) / 1)    # second constraint
# plot(l, 0*l) # l >= 0
# plot()


# legend(["constraint 1", "constraint 2", "nonnegative (l)", "nonnegative (s)", "objective"])
# xlabel(L"$l$")
# ylabel(L"$s$")
# title("Primal problem")
# grid()

ii)

In [6]:
using JuMP, Clp

m_dual = Model(with_optimizer(Clp.Optimizer))
@variable(m_dual, λ[1:2] >= 0)
@constraint(m_dual, 2λ[1] + 4λ[2] >= 8)
@constraint(m_dual, 3λ[1] + λ[2]  >= 5)
@objective(m_dual, Min, 160λ[1] + 200λ[2])

optimize!(m_dual)

display(m_dual)

println(termination_status(m_dual))
println("dual variables are: ", value.(λ))
println("Optimal objective is: ", objective_value(m))



A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Clp
Names registered in the model: λ

OPTIMAL
dual variables are: [1.1999999999999997, 1.4000000000000001]
Optimal objective is: 472.0
Coin0506I Presolve 2 (0) rows, 2 (0) columns and 4 (0) elements
Clp0006I 0  Obj 0 Primal inf 3.6666665 (2)
Clp0006I 2  Obj 472
Clp0000I Optimal - objective value 472
Clp0032I Optimal objective 472 - 2 iterations time 0.002
